In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import UnivariateSpline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

In [2]:
# 1. 65세 이상 노인 인구 데이터 (2015-2024)
years_elderly = np.array(range(2015, 2025))
elderly_population = np.array([
    12932,
    13255,
    13720,
    14123,
    14557,
    15080,
    15581,
    16265,
    16665,
    17216
])

# 2. 경로당 개수 데이터
years_senior_center = np.array([2017, 2019, 2020, 2021, 2024])
senior_center_count = np.array([295, 301, 305, 305, 313])  # 해당 연도의 경로당 개수

In [3]:
spline = UnivariateSpline(years_senior_center, senior_center_count, k=2, s=0)
senior_center_interpolated = spline(years_elderly)

# 3. 학습/테스트 데이터 분할
mask_train = years_elderly <= 2022
train_years = years_elderly[mask_train]
train_elderly = elderly_population[mask_train]
train_centers = senior_center_interpolated[mask_train]

mask_test = years_elderly >= 2023
test_years = years_elderly[mask_test]
test_elderly = elderly_population[mask_test]
test_centers = senior_center_interpolated[mask_test]

# 4. 다변량 다항 회귀: 연도 + 인구수 → 경로당 수
X_train = np.column_stack((train_years, train_elderly))
poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(X_train)
model = LinearRegression().fit(X_poly, train_centers)

# 5. 테스트 데이터 예측
X_test = np.column_stack((test_years, test_elderly))
X_test_poly = poly.transform(X_test)
pred_centers_test = model.predict(X_test_poly)

# 6. 노인 인구수 예측 모델 (연도 → 인구수)
poly_elderly = PolynomialFeatures(degree=2)
X_train_year_poly = poly_elderly.fit_transform(train_years.reshape(-1, 1))
model_elderly = LinearRegression().fit(X_train_year_poly, train_elderly)

# 7. 미래 예측 (2025~2030)
future_years = np.arange(2025, 2031)
X_future_year_poly = poly_elderly.transform(future_years.reshape(-1, 1))
future_elderly_pred = model_elderly.predict(X_future_year_poly)

X_future = np.column_stack((future_years, future_elderly_pred))
X_future_poly = poly.transform(X_future)
pred_centers_future = model.predict(X_future_poly)

# 8. 성능 평가 출력
print("\n=== 테스트 데이터 예측 결과 ===")
comparison_df = pd.DataFrame({
    'Year': test_years,
    'Actual_Elderly': test_elderly,
    'Predicted_Elderly': np.round(model_elderly.predict(poly_elderly.transform(test_years.reshape(-1, 1)))).astype(int),
    'Actual_Centers': np.round(test_centers).astype(int),
    'Predicted_Centers': np.round(pred_centers_test).astype(int),
})
print(comparison_df)

print("\n=== 테스트 데이터 성능 평가 ===")
print("  MAE:", mean_absolute_error(test_centers, pred_centers_test))
print("  RMSE:", np.sqrt(mean_squared_error(test_centers, pred_centers_test)))
print("  R²:", r2_score(test_centers, pred_centers_test))

# 9. 미래 예측 결과 출력
print("\n=== 미래 예측 (2025~2030) ===")
future_df = pd.DataFrame({
    'Year': future_years,
    'Predicted_Elderly': np.round(future_elderly_pred).astype(int),
    'Predicted_Centers': np.round(pred_centers_future).astype(int)
})
print(future_df)


=== 테스트 데이터 예측 결과 ===
   Year  Actual_Elderly  Predicted_Elderly  Actual_Centers  Predicted_Centers
0  2023           16665              16859             309                309
1  2024           17216              17532             313                310

=== 테스트 데이터 성능 평가 ===
  MAE: 1.7291970842648254
  RMSE: 2.175670490063652
  R²: 0.056006205405356946

=== 미래 예측 (2025~2030) ===
   Year  Predicted_Elderly  Predicted_Centers
0  2025              18246                310
1  2026              19000                315
2  2027              19796                325
3  2028              20631                341
4  2029              21508                365
5  2030              22425                401
